In [1]:
#Import statements
import pickle

In [2]:
with open('../../data/processed/full_set.pkl', 'rb') as f:
    full_set = pickle.load(f)
with open('../../data/processed/train_set.pkl', 'rb') as f:
    train_set = pickle.load(f)
with open('../../data/processed/test_set.pkl', 'rb') as f:
    test_set = pickle.load(f)
with open('../../data/processed/train_set_30.pkl', 'rb') as f:
    train_set_30 = pickle.load(f)
with open('../../data/processed/test_set_30.pkl', 'rb') as f:
    test_set_30 = pickle.load(f)
with open('../../data/processed/train_set_70.pkl', 'rb') as f:
    train_set_70 = pickle.load(f)
with open('../../data/processed/test_set_70.pkl', 'rb') as f:
    test_set_70 = pickle.load(f)

In [3]:
test_set_70.head()

,0,1,2,3,4,5,6,7,8,9,...,2294,2295,2296,2297,2298,2299,2300,2301,2302,target
7217,42.0,38.0,33.0,34.0,43.0,58.0,53.0,46.0,39.0,38.0,...,25.0,23.0,26.0,27.0,28.0,26.0,24.0,21.0,21.0,7
8291,16.0,17.0,20.0,26.0,36.0,46.0,47.0,44.0,40.0,36.0,...,15.0,16.0,27.0,52.0,74.0,87.0,91.0,90.0,88.0,8
4607,39.0,38.0,27.0,59.0,82.0,78.0,41.0,15.0,15.0,13.0,...,23.0,31.0,39.0,48.0,41.0,26.0,27.0,37.0,29.0,4
5114,17.0,16.0,15.0,15.0,16.0,17.0,15.0,15.0,16.0,17.0,...,14.0,15.0,14.0,15.0,15.0,15.0,14.0,12.0,13.0,5
1859,22.0,21.0,20.0,20.0,20.0,21.0,21.0,20.0,19.0,18.0,...,38.0,28.0,23.0,19.0,17.0,18.0,18.0,19.0,19.0,1


# Linear Classifier

In [ ]:
#DO ZE CODE

# Multilayer Perceptron

In [4]:
#DO MORE OF ZE CODE

# 10-fold cross validation

In [ ]:
#DO MORE MORE OF ZE CODE

# Visualize

In [ ]:
#DO MORE MORE MORE OF ZE CODE